In [1]:
import torch
import numpy as np
import pandas as pd
from torch import nn

In [2]:
# Dataset from https://www.kaggle.com/datasets/saurabhshahane/electricity-load-forecasting
df = pd.read_csv('kaggle_electricity_load_forecasting_train.csv')
df = df.set_index('datetime')

In [3]:
data_output = np.array(df['nat_demand']).reshape(-1, 1)
data_input = np.array(df[['T2M_toc', 'QV2M_toc','TQL_toc','W2M_toc']])

In [4]:
data_output = torch.Tensor(data_output).cuda()
data_input = torch.Tensor(data_input).cuda()

In [5]:
print(data_output.shape, data_input.shape)

torch.Size([43775, 1]) torch.Size([43775, 4])


In [37]:
class AttentionNet(nn.Module):
    def __init__(self, inp_size, out_size, linear_size=36):
        super(AttentionNet, self).__init__()

        self.inp_size = inp_size
        self.linear_size = linear_size
        self.out_size = out_size
        
        self.inp_lin = nn.Linear(self.inp_size, self.linear_size)
        self.lin1 = nn.Linear(self.linear_size, self.linear_size)
        self.lin2 = nn.Linear(self.linear_size, self.linear_size)
        self.out_lin = nn.Linear(self.linear_size, self.out_size)
    def forward(self, x):
        x = self.inp_lin(x)
        x1 = self.lin1(x)
        x2 = self.lin2(x)
        temp = torch.matmul(x1, torch.matmul(x2.transpose(-2, -1), x2))
        x = self.out_lin(temp)
        return x


class DLFTNet(nn.Module):
    def __init__(self, inp_size, out_size, linear_size=36):
        super(DLFTNet, self).__init__()

        self.inp_size = inp_size
        self.linear_size = linear_size
        self.out_size = out_size
        
        self.inp_lin = nn.Linear(self.inp_size, self.linear_size)
        self.lin1 = nn.Linear(self.linear_size, self.linear_size)
        self.out_lin = nn.Linear(self.linear_size ** 2, self.out_size)
        
    def forward(self, x):
        x = self.inp_lin(x)
        x1 = torch.fft.fft(x).real.view(-1, x.shape[1], 1)
        x2 = self.lin1(x).view(-1, 1, x.shape[1])
        temp = torch.matmul(x1, x2).reshape(-1, x.shape[1] ** 2)
        x = self.out_lin(temp)
        return x

class MLPNet(nn.Module):
    def __init__(self, inp_size, out_size, freq_domain=None, linear_size=36):
        super(MLPNet, self).__init__()

        self.inp_size = inp_size
        self.linear_size = linear_size
        self.out_size = out_size
        self.freq_domain = freq_domain
        
        self.inp_lin = nn.Linear(self.inp_size, self.linear_size)
        self.lin1 = nn.Linear(self.linear_size, self.linear_size)
        self.out_lin = nn.Linear(self.linear_size, self.out_size)
        
    def forward(self, x):
        x = self.inp_lin(x)
        x1 = self.lin1(x)
        x = self.out_lin(x1)
        return x

In [19]:
net = MLPNet(4, 1).cuda()
opti = torch.optim.Adam(net.parameters(), lr=0.0001)
loss = torch.nn.L1Loss().cuda()

for i in range(0, 10000) :
    opti.zero_grad()
    out = net(data_input)
    l = loss(out, data_output)
    l.backward()
    opti.step()
    if i % 100 == 0 :
        print(i, l.item())

0 1184.826416015625


KeyboardInterrupt: 

In [12]:
net = AttentionNet(4, 1).cuda()
opti = torch.optim.Adam(net.parameters(), lr=0.0001)
loss = torch.nn.L1Loss().cuda()

for i in range(0, 10000) :
    opti.zero_grad()
    out = net(data_input)
    l = loss(out, data_output)
    l.backward()
    opti.step()
    if i % 100 == 0 :
        print(i, l.item())

0 13934369.0
1000 7959.78125
2000 6174.310546875
3000 4901.47265625
4000 3565.36767578125
5000 2082.5166015625
6000 1295.8763427734375
7000 576.42822265625
8000 165.900634765625
9000 115.19146728515625


In [38]:
net = DLFTNet(4, 1).cuda()
opti = torch.optim.Adam(net.parameters(), lr=0.0001)
loss = torch.nn.L1Loss().cuda()

for i in range(0, 10000) :
    opti.zero_grad()
    out = net(data_input)
    l = loss(out, data_output)
    l.backward()
    opti.step()
    if i % 100 == 0 :
        print(i, l.item())

0 1133.9453125
1000 114.64875030517578
2000 114.60354614257812
3000 114.53772735595703
4000 114.44965362548828


KeyboardInterrupt: 